In [2]:
#langchian components for use:

from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

#support for dataset retrieval with hugging face
from datasets import load_dataset



c:\Users\prath\Downloads\bootcamp\Project-63\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

!pip install cassio

In [4]:
import cassio

In [5]:
from PyPDF2 import PdfReader

In [6]:
ASTRA_DB_APPLICATION_TOKEN='AstraCS:ZJtQGHJLBkDQeswNzJPWkfQA:17fd287fb87c82d90dfe1462aeffdd0688dbfa3aa226c4400f634fb3d5c29759'
ASTRA_DB_ID='0d2c96fc-3705-479a-88b0-dbd9fc29102c'
OPENAI_API_KEY='sk-ThTby-QVVkzrVhR6Oh67q_CcNu1jkpd2dUBmXnHPPoT3BlbkFJHqe8DpL5oXR2kespPeFvTYWdy7PI8CxyxE9hgjPDIA'


In [7]:
#path to pdf
pdfreader=PdfReader('Budget_Speech.pdf')

In [8]:
from typing_extensions import Concatenate
raw_text=''
for i,page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text+=content

In [9]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN,database_id=ASTRA_DB_ID)

In [10]:
llm=OpenAI(openai_api_key=OPENAI_API_KEY)
embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

C:\Users\prath\AppData\Local\Temp\ipykernel_23536\3645335011.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm=OpenAI(openai_api_key=OPENAI_API_KEY)
C:\Users\prath\AppData\Local\Temp\ipykernel_23536\3645335011.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [11]:
astra_vector_store=Cassandra(
    embedding=embedding,
    table_name='qa_mini_demo',
    session=None,
    keyspace=None,
)

In [12]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter=CharacterTextSplitter(
    separator='\n',
    chunk_size=800,
    length_function=len,
)

texts=text_splitter.split_text(raw_text)

In [13]:
texts[:50]

['GOVERNMENT OF INDIA\nBUDGET 2024-2025\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nJuly 23,  2024 \nCONTENTS  \n \nPART – A \n Page No.  \nIntroduction 1 \nGlobal Context  1 \nInterim Budget  2 \nBudget Theme  2 \nBudget Priorities  2 \n(i) Productivity and resilience in Agriculture   \n(ii) Employment & Skilling  \n(iii) Inclusive Human Resource Development and Social Justice   \n(iv) Manufacturing & Services  \n(v) Urban Development   \n(vi) Energy Security  \n(vii) Infrastructure  \n(viii) Innovation, Research & Development  \n(ix) Next Generation Reforms  \nBudget Estimates 2024- 25 20 \n \n \nPART – B \nIndirect taxes  22 \nDirect Taxes   25 \n Annexure to Part -A 31 \nAnnexure to Part -B 36 \n \n   \n \nBudget 2024 -2025  \n \nSpeech of  \nNirmala Sitharaman  \nMinister of Finance  \nJuly 23 , 2024',
 'PART – B \nIndirect taxes  22 \nDirect Taxes   25 \n Annexure to Part -A 31 \nAnnexure to Part -B 36 \n \n   \n \nBudget 2024 -2025  \n \nSpeech of  \nNirmala Sitharaman

In [14]:
astra_vector_store.add_texts(texts[:50])
print('inserted %i headlines.'%len(texts[:50]))
astra_vector_index=VectorStoreIndexWrapper(vectorstore=astra_vector_store)

inserted 50 headlines.


In [21]:
first_question=True
while True:
    if first_question:
        query_text=input("\nEnter your quesstion(or type 'quit' or exit:)").strip()
    else:
        query_text=input("\n what is your next question(or type 'quit' or exit:)").strip()

    if query_text.lower() =='quit':
        break
    if query_text=="":
        continue
    first_question=False
    print('\nQuestion:\"%s\''%query_text)
    answer=astra_vector_index.query(query_text,llm=llm).strip()
    print('Answer:\'%s\'\n' %answer)

    print('First Documents by relevance')
    for doc,score in astra_vector_store.similarity_search_with_score(query_text,k=4):
        print(" [%0.4f] \"%s...\"" %(score,doc.page_content[:84]))




Question:"how much is the agriculture target will be increased and what the focus will be'
Answer:'I do not know the specific target for agriculture or what the focus will be. The context provided does not mention any specific numbers or targets.'

First Documents by relevance
 [0.9250] "made announcements with an intent to strengthen them and step up their implementatio..."
 [0.9191] "Release of new varieties  
11. New 109 high -yielding and climate -resilient varieti..."
 [0.9109] "strengthen their production, storage and marketing. As announced in the 
interim bud..."
 [0.9078] "Digital Public Infrastructure for Agriculture  
15. Buoyed by the success of the pil..."
